# Análise exploratória

## Descrição do notebook

O objetivo deste notebook é obter uma visão geral dos dados sobre violência contra a mulher, nos estados do Pará e Paraná.

## Sumário

1. [Importação dos módulos e pacotes necessários](#importacao-dos-modulos-e-pacotes-necessarios)
2. [Carregamento dos dados](#carregamento-dos-dados)
3. [Análises](#analises)

----

In [ ]:
# Pandas (https://pandas.pydata.org/)
import pandas as pd

# Locale (https://docs.python.org/3/library/locale.html)
import locale # Função de localização para formatar datas, número etc.

# Datetime (https://docs.python.org/3/library/datetime.html)
import datetime as dt # Manipulação de data

# Numpy (https://numpy.org/)
import numpy as np

# Bibliotecas para gráficos (https://seaborn.pydata.org/)
# %matplotlib inline
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
# Define a localização para português do Brasil
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')

---

## Carregamento dos dados <a id="carregamento-dos-dados"></a>

In [ ]:
df = pd.read_csv('./dados-tratados/20240603152915-dados_unificados_VCM.csv', sep='|')
df.head()

In [ ]:
df.columns

In [ ]:
# Cria uma cópia de segurança do df carregado
df_bkp = df

In [ ]:
# Converte as colunas necessário para o de data
df['data_registro'] = pd.to_datetime(df['data_registro'])
df['hora_registro'] = pd.to_datetime(df['hora_registro'])
df['data_fato'] = pd.to_datetime(df['data_fato'])
df['hora_fato'] = pd.to_datetime(df['hora_fato'])
df['vit_dt_nasc'] = pd.to_datetime(df['vit_dt_nasc'])
df['aut_dt_nasc'] = pd.to_datetime(df['aut_dt_nasc'])

In [ ]:
# Número total de linhas no dataframe
len(df)

In [ ]:
# Número total de linhas sem a informação da identificação do boletim, ou seja, os registros do estado do Pará
df['id_boletim'].isnull().sum()

In [ ]:
# Completa os valores nulos pelo valor do índice. Assim, cada linha do estado do Pará terá uma identificação.
df['id_boletim'] = df['id_boletim'].fillna(pd.Series(df.index))

In [ ]:
# Verifica que não restou nenhuma linha sem identificação
df['id_boletim'].isnull().sum()

In [ ]:
# Número total de casos
ocorrencias = df.groupby('id_boletim')
len(ocorrencias)

In [ ]:
# Mantendo a primeira ocorrência
ocorrencias_unicas = ~df.duplicated(subset='id_boletim', keep="first")
df_ocorrencias = df[ocorrencias_unicas]
len(df_ocorrencias)

In [ ]:
df_ocorrencias

## Análises <a id="analises"></a>

### Intervalo entre ocorrência e registro

In [ ]:
pd.options.mode.copy_on_write = True

In [ ]:
# Verificação de quantos registros não são feitos na mesma data da ocorrência
print(len(df_ocorrencias[df_ocorrencias['data_fato'] != df_ocorrencias['data_registro']]))

In [ ]:
# Convertendo os campos 'data_registro' e 'data_fato' para o formato de data
df_ocorrencias['data_registro'] = pd.to_datetime(df_ocorrencias['data_registro'])
df_ocorrencias['data_fato'] = pd.to_datetime(df_ocorrencias['data_fato'])

In [ ]:
# Cria uma coluna para o intervalo, em dias, entre a data da ocorrência e do registro
df_ocorrencias['intervalo_fato_registro'] = (df_ocorrencias['data_registro'] - df_ocorrencias['data_fato']).dt.days

In [ ]:
# Estatísticas descritivas desse intervalo
df_ocorrencias['intervalo_fato_registro'].describe()

In [ ]:
# Distribuição dessa variável intervalo
df_ocorrencias['intervalo_fato_registro'].value_counts()

In [ ]:
# Verifica quantas vezes o registro foi realizado na mesma data do fato
print(df_ocorrencias['intervalo_fato_registro'].value_counts().get(0, 0))
# Verifica o percentual de quantas vezes isso aconteceu
print(df_ocorrencias['intervalo_fato_registro'].value_counts().get(0, 0)/len(df_ocorrencias))

#### Distribuição do intervalo

In [ ]:
df_intervalo = df_ocorrencias.intervalo_fato_registro.value_counts().reset_index().rename(columns={'intervalo_fato_registro': 'Intervalo', 'count': 'Total'}).sort_values(by='Intervalo').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_intervalo['Porcentagem'] = df_intervalo.Total / df_intervalo.Total.sum()

df_intervalo.head(20).style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Data do fato

In [ ]:
# Selecionando o registro com o fato mais antigo
fato_antigo = df_ocorrencias.iloc[df_ocorrencias['data_fato'].argmin()]
fato_antigo
print(f"O fato mais ANTIGO da base ocorreu em {fato_antigo['data_fato'].strftime('%d de %B de %Y')}.")

# Selecionando o registro com o fato mais recente
fato_recente = df_ocorrencias.iloc[df_ocorrencias['data_fato'].argmax()]
fato_recente
print(f"O fato mais RECENTE da base ocorreu em {fato_recente['data_fato'].strftime('%d de %B de %Y')}.")

### Anos

In [ ]:
# Cria um novo dataframe para guardar os dados dos registros por ano
df_ano_fato = df_ocorrencias['data_fato'].dt.year.value_counts().reset_index().rename(columns={'data_fato': 'Ano', 'count': 'Total'}).sort_values(by='Ano').reset_index(drop=True)

# Criando uma coluna com a porcentagem dos valores anuais
df_ano_fato['Porcentagem'] = df_ano_fato.Total / df_ano_fato.Total.sum()

In [ ]:
# Exibe a tabela
df_ano_fato.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
df_ano_fato['Total'].describe()

In [ ]:
# Imprime o gráfico com os fatos distribuídos pelos anos da base
# Define o estilo do gráfico
sns.set_theme(style="whitegrid")
# Define o tamanho do gráfico em polegadas
fig = plt.gcf()
fig.set_size_inches(10, 4)
# Cria um novo gráfico de barras
#  palette="viridis", 
g = sns.barplot(x=df_ano_fato.Ano, y=df_ano_fato.Total, alpha=0.9, color='#666666')
plt.title('Distribuição de fatos por ano', fontsize=16, pad=20)

# Rótulos dos eixos
plt.ylabel('Fatos', fontsize=12, labelpad=10)
plt.xlabel('Anos', fontsize=12, labelpad=10)
plt.xticks(rotation=90)
# Adiciona os valores às colunas
for i in range(0, len(df_ano_fato.Total)):
    g.text(i, df_ano_fato.Total[i]+2, int(df_ano_fato.Total[i]), color='black', ha="center", fontsize=10)
# Exibe a figura
plt.show()

Verificação da distribuição dos fatos pelos anos, separando os dois estados.

In [ ]:
df_pa = df_ocorrencias[df_ocorrencias['estado'] == 'PA']
df_pr = df_ocorrencias[df_ocorrencias['estado'] == 'PR']

# Cria um novo dataframe para guardar os dados dos registros por ano
df_ano_fato_pa = df_pa['data_fato'].dt.year.value_counts().reset_index().rename(columns={'data_fato': 'Ano', 'count': 'Total'}).sort_values(by='Ano').reset_index(drop=True)
df_ano_fato_pr = df_pr['data_fato'].dt.year.value_counts().reset_index().rename(columns={'data_fato': 'Ano', 'count': 'Total'}).sort_values(by='Ano').reset_index(drop=True)

# Criando uma coluna com a porcentagem dos valores anuais
df_ano_fato_pa['Porcentagem'] = df_ano_fato_pa.Total / df_ano_fato_pa.Total.sum()
df_ano_fato_pr['Porcentagem'] = df_ano_fato_pr.Total / df_ano_fato_pr.Total.sum()

# Exibe a tabela
df_ano_fato_pa.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
df_ano_fato_pr.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Meses

In [ ]:
df_ocorrencias['mes_fato'].value_counts()

In [ ]:
# Conta os fatos por ano
# contagem_por_mes = df['mes_fato'].value_counts()

# Cria um novo dataframe para guardar os dados dos registros por mês
df_mes_fato = df_ocorrencias['mes_fato'].value_counts().reset_index().rename(columns={'mes_fato': 'Mês', 'count': 'Total'}).sort_values(by='Total').reset_index(drop=True)

# Importa um novo pacote para trabalhar a coluna 'Mês' como categoria
from pandas.api.types import CategoricalDtype

# Define a ordem correta dos meses
cat_month_order = CategoricalDtype(
    ['JANEIRO', 'FEVEREIRO', 'MARCO', 'ABRIL', 'MAIO', 'JUNHO', 'JULHO', 'AGOSTO', 'SETEMBRO', 'OUTUBRO', 'NOVEMBRO', 'DEZEMBRO'], 
    ordered=True
)
# Define que o tipo de dado contido na coluna 'Mês' como categoria
#df_mes_fato['Mês'] = df_mes_fato['Mês'].astype(cat_month_order)
# Ordena os dados
df_mes_fato.sort_values('Mês', inplace=True)

df_mes_fato.reset_index(drop=True, inplace=True)
df_mes_fato

# Criando uma coluna com a porcentagem dos valores dos meses
df_mes_fato['Porcentagem'] = df_mes_fato.Total / df_mes_fato.Total.sum()

# Exibe a tabela
df_mes_fato.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Imprime o gráfico com os fatos distribuídos pelos meses da base
# Define o estilo do gráfico
sns.set_theme(style="whitegrid")
# Define o tamanho do gráfico em polegadas
fig = plt.gcf()
fig.set_size_inches(10, 4)
# Cria um novo gráfico de barras
#  palette="viridis", 
g = sns.barplot(x=df_mes_fato['Mês'], y=df_mes_fato.Total, alpha=0.9, color='#666666')
plt.title('Distribuição de fatos por mês', fontsize=16, pad=20)

# Rótulos dos eixos
plt.ylabel('Fatos', fontsize=12, labelpad=10)
plt.xlabel('Meses', fontsize=12, labelpad=10)
plt.xticks(rotation=90)
# Adiciona os valores às colunas
for i in range(0, len(df_mes_fato.Total)):
    g.text(i, df_mes_fato.Total[i]+2, int(df_mes_fato.Total[i]), color='black', ha="center", fontsize=10)
# Exibe a figura
plt.show()

### Dia da semana

In [ ]:
df_ocorrencias['dia_semana']

In [ ]:
# Verificação dos horários dos datos
df_semana = df_ocorrencias.dia_semana.value_counts().reset_index().rename(columns={'dia_semana': 'Dia', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_semana['Porcentagem'] = df_semana.Total / df_semana.Total.sum()

# Define a ordem correta dos meses
cat_day_order = CategoricalDtype(
    ['SEG', 'TER', 'QUA', 'QUI', 'SEX', 'SAB', 'DOM'], 
    ordered=True
)
# Define que o tipo de dado contido na coluna 'Mês' como categoria
df_semana['Dia'] = df_semana['Dia'].astype(cat_day_order)

df_semana.sort_values('Dia', inplace=True)

df_semana.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Horário dos fatos

In [ ]:
# Verificação dos horários dos datos
df_horario = df_ocorrencias.faixa_horaria.value_counts().reset_index().rename(columns={'faixa_horaria': 'Faixa', 'count': 'Total'}).sort_values(by='Faixa').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_horario['Porcentagem'] = df_horario.Total / df_horario.Total.sum()

# Define a ordem correta dos meses
cat_hour_order = CategoricalDtype(
    ['Manhã', 'Tarde', 'Noite', 'Madrugada'],
    ordered=True
)
# Define que o tipo de dado contido na coluna 'Mês' como categoria
df_horario['Faixa'] = df_horario['Faixa'].astype(cat_hour_order)

df_horario.sort_values('Faixa', inplace=True)

df_horario.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Verificação dos horários dos datos
df_horario = df_ocorrencias.faixa_horaria_2h.value_counts().reset_index().rename(columns={'faixa_horaria_2h': 'Faixa', 'count': 'Total'}).sort_values(by='Faixa').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_horario['Porcentagem'] = df_horario.Total / df_horario.Total.sum()

df_horario.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Características da vítima

In [ ]:
# Completa os valores nulos pelo valor do índice. Assim, cada linha do estado do Pará terá uma identificação.
df['vitima'] = df['vitima'].fillna(pd.Series(df.index))

In [ ]:
df.vitima

In [ ]:
# Mantendo a primeira vítima
vitimas_unicas = ~df.duplicated(subset='vitima', keep="first")
df_vitimas = df[vitimas_unicas]
len(df_vitimas)

##### Idade

In [ ]:
# Remoção dos registros cuja idade é superior a 120 ou estão com o dado em branco
idade = df_vitimas[(df_vitimas['vit_idade'] < 120) & (df_vitimas['vit_idade'] != '')]['vit_idade']
print(f"Há {len(idade)} registros com o dado de idade.")

In [ ]:
# Estatísticas descritivas da idade
idade.describe()

In [ ]:
print(idade.mode())
print(idade.median())

In [ ]:
# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 11, 17, 24, 29, 34, 39, 44, 49, 64, 105]
labels = ['De 0 a 11 anos', 'De 12 a 17 anos', 'De 18 a 24 anos', 'De 25 a 29 anos', 'De 30 a 34 anos', 'De 35 a 39 anos', 'De 40 a 44 anos', 'De 45 a 49 anos', 'De 50 a 64 anos', 'Mais de 65 anos']
categories = labels + ['Desconhecida']
df_vitimas['vit_faixa_etaria_2'] = pd.cut(df_vitimas['vit_idade'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
df_vitimas['vit_faixa_etaria_2'] = pd.Categorical(df_vitimas['vit_faixa_etaria_2'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
df_vitimas['vit_faixa_etaria_2'] = df_vitimas['vit_faixa_etaria_2'].fillna('Desconhecida')

df_vitimas['vit_faixa_etaria_2'].value_counts()

In [ ]:
# Verificação dos horários dos datos
df_vit_faixa_etaria = df_vitimas.vit_faixa_etaria_2.value_counts().reset_index().rename(columns={'vit_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_vit_faixa_etaria['Porcentagem'] = df_vit_faixa_etaria.Total / df_vit_faixa_etaria.Total.sum()

df_vit_faixa_etaria.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Verificação dos horários dos datos
df_vit_faixa_etaria_pa = df_vitimas[df_vitimas['estado'] == 'PA'].vit_faixa_etaria_2.value_counts().reset_index().rename(columns={'vit_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_vit_faixa_etaria_pa['Porcentagem'] = df_vit_faixa_etaria_pa.Total / df_vit_faixa_etaria_pa.Total.sum()

df_vit_faixa_etaria_pa.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Verificação dos horários dos datos
df_vit_faixa_etaria_pr = df_vitimas[df_vitimas['estado'] == 'PR'].vit_faixa_etaria_2.value_counts().reset_index().rename(columns={'vit_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_vit_faixa_etaria_pr['Porcentagem'] = df_vit_faixa_etaria_pr.Total / df_vit_faixa_etaria_pr.Total.sum()

df_vit_faixa_etaria_pr.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

#### Cor da pele

In [ ]:
# Cria um novo dataframe para guardar os dados dos registros por ano
df_vit_cor_pele_pa = df_pa['vit_cor_pele'].value_counts().reset_index().rename(columns={'vit_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)
df_vit_cor_pele_pr = df_pr['vit_cor_pele'].value_counts().reset_index().rename(columns={'vit_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)

# Criando uma coluna com a porcentagem dos valores anuais
df_vit_cor_pele_pa['Porcentagem'] = df_vit_cor_pele_pa.Total / df_vit_cor_pele_pa.Total.sum()
df_vit_cor_pele_pr['Porcentagem'] = df_vit_cor_pele_pr.Total / df_vit_cor_pele_pr.Total.sum()

# Exibe a tabela
df_vit_cor_pele_pa.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
df_vit_cor_pele_pr.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

##### Nova versão

In [ ]:
vitima_sem_dados_cor_de_pele = df_vitimas['vit_cor_pele'].isnull().sum()
n_vitimas = len(df_vitimas)

In [ ]:
print(f'De um total de {n_vitimas} registradas, {vitima_sem_dados_cor_de_pele} não apresentam dado referem à cor de pele.')

In [ ]:
# Remoção dos registros cuja cor de pele não existe
vit_cor_pele = df_vitimas.dropna(subset=['vit_cor_pele'])['vit_cor_pele']
print(f"Há {len(vit_cor_pele)} registros com o dado de idade.")

In [ ]:
vit_cor_pele.value_counts()

In [ ]:
df_vit_cor_pele_geral = df_vitimas['vit_cor_pele'].value_counts().reset_index().rename(columns={'vit_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)
df_vit_cor_pele_geral['Porcentagem'] = df_vit_cor_pele_geral.Total / df_vit_cor_pele_geral.Total.sum()
df_vit_cor_pele_geral.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Dados do PA
vit_cor_pele_pa = df_vitimas[df_vitimas['estado'] == 'PA'].dropna(subset=['vit_cor_pele'])['vit_cor_pele']
vit_cor_pele_pa.value_counts()

In [ ]:
# Dados do PR
vit_cor_pele_pr = df_vitimas[df_vitimas['estado'] == 'PR'].dropna(subset=['vit_cor_pele'])['vit_cor_pele']
vit_cor_pele_pr.value_counts()

Carregando os dados originais do Pará para verificar essa variável.

In [ ]:
# Carregamento dos dados utilizando o arquivo XLSX e pandas
dados_pa = pd.read_csv('./dados/PA/raw_BD_VD_2010a2022_PA.csv', sep='@')
dados_pa['vit_cor_pele'].value_counts()

#### Grau de instrução

In [ ]:
df_vitimas.columns

In [ ]:
df_vitimas['vit_grau_inst'].isnull().sum()

In [ ]:
df_vitimas['vit_grau_inst'].value_counts()

In [ ]:
substituicoes = {
    'PRIMEIRO GRAU INCOMPLETO': 'ENSINO FUNDAMENTAL INCOMPLETO',
    'PRIMEIRO GRAU COMPLETO': 'ENSINO FUNDAMENTAL COMPLETO',
    'SEGUNDO GRAU INCOMPLETO': 'ENSINO MEDIO INCOMPLETO',
    'SEGUNDO GRAU COMPLETO': 'ENSINO MEDIO COMPLETO',
    'TERCEIRO GRAU INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO',
    'TERCEIRO GRAU COMPLETO': 'ENSINO SUPERIOR COMPLETO',
    'SUPERIOR INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO'
}

df_vitimas['vit_grau_inst'] = df_vitimas['vit_grau_inst'].replace(substituicoes)



In [ ]:
df_vitimas['vit_grau_inst'].value_counts()

In [ ]:
df_vit_grau_inst = df_vitimas['vit_grau_inst'].value_counts().reset_index().rename(columns={'vit_grau_inst': 'Grau de instrução', 'count': 'Total'}).sort_values(by='Grau de instrução').reset_index(drop=True)
df_vit_grau_inst['Porcentagem'] = df_vit_grau_inst.Total / df_vit_grau_inst.Total.sum()
df_vit_grau_inst.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

#### Profissão

In [ ]:
df_vitimas['vit_profissao'].isnull().sum()

In [ ]:
df_vitimas['vit_profissao'].notnull().sum()

In [ ]:
9684+15840

In [ ]:
df_vitimas['vit_profissao'].value_counts()

In [ ]:
# Tornando como padrão os termos com mais registros
substituicoes = {
    'ADMINISTRADOR': 'ADMINISTRADOR DE EMPRESA',
    'ADVOGADO(A)': 'ADVOGADO',
    'AGRICULTOR(A)': 'AGRICULTOR',
    'APOSENTADO(A)': 'APOSENTADO',
    'CABELEIREIRO': 'CABELEIREIRA',
    'CAMAREIRO': 'CAMAREIRA',
    'COSTUREIRO(A),': 'COSTUREIRO',
    'COZINHEIRO(A),': 'COZINHEIRO',
    'ESTUDAMTE,': 'ESTUDANTE',
    'LAVRADOR(A)': 'LAVRADOR',
    'PEDAGOGO(A)': 'PEDAGOGA',
    'PEDAGOGO': 'PEDAGOGA',
    'PROMOTOR DE VENDAS': 'PROMOTORA DE VENDAS',
    'SECRETARIO(A)': 'SECRETARIA',
    'VENDEDOR(A)': 'VENDEDOR'
}

df_vitimas['vit_profissao'] = df_vitimas['vit_profissao'].replace(substituicoes)



In [ ]:
df_vit_profissao = df_vitimas['vit_profissao'].value_counts().reset_index().rename(columns={'vit_profissao': 'Profissão', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
df_vit_profissao['Porcentagem'] = df_vit_profissao.Total / df_vit_profissao.Total.sum()
df_vit_profissao.head(30).style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Características do autor

In [ ]:
df.columns

In [ ]:
# Preenchendo com uma ID os autores do estado do Pará que possuem alguma informação sobre o grau de relacionamento ou sobre a idade
df['autor'] = df[(df['estado'] == 'PA') & ((df['grau_de_relacionamento'].notnull()) & (df['grau_de_relacionamento'] != 'PREJUDICADO')) | (df['aut_idade'].notnull())]['autor'].fillna(pd.Series(df.index))

In [ ]:
# Mantendo o primeiro autor
autores_unicos = ~df.duplicated(subset='autor', keep="first")
df_autores = df[autores_unicos]
len(df_autores)

In [ ]:
len(df_autores)/len(df_ocorrencias)

In [ ]:
df_autores['grau_de_relacionamento'].value_counts()

#### Idade

In [ ]:
len(df_autores)

In [ ]:
# Remoção dos registros cuja idade é superior a 120 ou estão com o dado em branco
idade = df_autores[(df_autores['aut_idade'] < 120) & (df_autores['aut_idade'] != '')]['aut_idade']
print(f"Há {len(idade)} registros com o dado de idade.")

In [ ]:
# Estatísticas descritivas da idade
idade.describe()

In [ ]:
print(idade.mode())
print(idade.median())

In [ ]:
# Definindo os intervalos e os rótulos das faixas etárias
intervals = [0, 11, 17, 24, 29, 34, 39, 44, 49, 64, 105]
labels = ['De 0 a 11 anos', 'De 12 a 17 anos', 'De 18 a 24 anos', 'De 25 a 29 anos', 'De 30 a 34 anos', 'De 35 a 39 anos', 'De 40 a 44 anos', 'De 45 a 49 anos', 'De 50 a 64 anos', 'Mais de 65 anos']
categories = labels + ['Desconhecida']
df_autores['aut_faixa_etaria_2'] = pd.cut(df_autores['aut_idade'], bins=intervals, labels=labels, right=False)

# Adiciona a categoria 'Desconhecida' à lista de categorias da faixa etária
df_autores['aut_faixa_etaria_2'] = pd.Categorical(df_autores['aut_faixa_etaria_2'], categories=categories)
# Preenche os registros com valores nulos como 'Desconhecida'
df_autores['aut_faixa_etaria_2'] = df_autores['aut_faixa_etaria_2'].fillna('Desconhecida')

df_autores['aut_faixa_etaria_2'].value_counts()

In [ ]:
# Verificação dos horários dos datos
df_aut_faixa_etaria = df_autores.aut_faixa_etaria_2.value_counts().reset_index().rename(columns={'aut_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_aut_faixa_etaria['Porcentagem'] = df_aut_faixa_etaria.Total / df_aut_faixa_etaria.Total.sum()

df_aut_faixa_etaria.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Verificação dos horários dos datos
df_aut_faixa_etaria_pa = df_autores[df_autores['estado'] == 'PA'].aut_faixa_etaria_2.value_counts().reset_index().rename(columns={'aut_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_aut_faixa_etaria_pa['Porcentagem'] = df_aut_faixa_etaria_pa.Total / df_aut_faixa_etaria_pa.Total.sum()

df_aut_faixa_etaria_pa.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Verificação dos horários dos datos
df_aut_faixa_etaria_pr = df_autores[df_autores['estado'] == 'PR'].aut_faixa_etaria_2.value_counts().reset_index().rename(columns={'aut_faixa_etaria_2': 'Faixa etária', 'count': 'Total'}).sort_values(by='Faixa etária').reset_index(drop=True)
# Criando uma coluna com a porcentagem dos valores
df_aut_faixa_etaria_pr['Porcentagem'] = df_aut_faixa_etaria_pr.Total / df_aut_faixa_etaria_pr.Total.sum()

df_aut_faixa_etaria_pr.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

#### Cor da pele

In [ ]:
# Cria um novo dataframe para guardar os dados dos registros por ano
df_aut_cor_pele_pa = df_pa['aut_cor_pele'].value_counts().reset_index().rename(columns={'aut_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)
df_aut_cor_pele_pr = df_pr['aut_cor_pele'].value_counts().reset_index().rename(columns={'aut_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)

# Criando uma coluna com a porcentagem dos valores anuais
df_aut_cor_pele_pa['Porcentagem'] = df_aut_cor_pele_pa.Total / df_aut_cor_pele_pa.Total.sum()
df_aut_cor_pele_pr['Porcentagem'] = df_aut_cor_pele_pr.Total / df_aut_cor_pele_pr.Total.sum()

# Exibe a tabela
df_aut_cor_pele_pa.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
df_aut_cor_pele_pr.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

##### Nova versão

In [ ]:
autor_sem_dados_cor_de_pele = df_autores['aut_cor_pele'].isnull().sum()
n_autores = len(df_autores)

In [ ]:
print(f'De um total de {n_autores} registrados, {autor_sem_dados_cor_de_pele} não apresentam dado referem à cor de pele.')

In [ ]:
# Remoção dos registros cuja cor de pele não existe
aut_cor_pele = df_autores.dropna(subset=['aut_cor_pele'])['aut_cor_pele']
print(f"Há {len(aut_cor_pele)} registros com o dado de cor de pele.")

In [ ]:
aut_cor_pele.value_counts()

In [ ]:
df_aut_cor_pele_geral = df_autores['aut_cor_pele'].value_counts().reset_index().rename(columns={'aut_cor_pele': 'Cor da pele', 'count': 'Total'}).sort_values(by='Cor da pele').reset_index(drop=True)
df_aut_cor_pele_geral['Porcentagem'] = df_aut_cor_pele_geral.Total / df_aut_cor_pele_geral.Total.sum()
df_aut_cor_pele_geral.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
# Dados do PA
aut_cor_pele_pa = df_autores[df_autores['estado'] == 'PA'].dropna(subset=['aut_cor_pele'])['aut_cor_pele']
aut_cor_pele_pa.value_counts()

In [ ]:
# Dados do PR
aut_cor_pele_pr = df_autores[df_autores['estado'] == 'PR'].dropna(subset=['aut_cor_pele'])['aut_cor_pele']
aut_cor_pele_pr.value_counts()

#### Grau de instrução

In [ ]:
df_autores.columns

In [ ]:
df_autores['aut_grau_inst'].isnull().sum()

In [ ]:
df_autores['aut_grau_inst'].value_counts()

In [ ]:
substituicoes = {
    'PRIMEIRO GRAU INCOMPLETO': 'ENSINO FUNDAMENTAL INCOMPLETO',
    'PRIMEIRO GRAU COMPLETO': 'ENSINO FUNDAMENTAL COMPLETO',
    'SEGUNDO GRAU INCOMPLETO': 'ENSINO MEDIO INCOMPLETO',
    'SEGUNDO GRAU COMPLETO': 'ENSINO MEDIO COMPLETO',
    'TERCEIRO GRAU INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO',
    'TERCEIRO GRAU COMPLETO': 'ENSINO SUPERIOR COMPLETO',
    'SUPERIOR INCOMPLETO': 'ENSINO SUPERIOR INCOMPLETO'
}

df_autores['aut_grau_inst'] = df_autores['aut_grau_inst'].replace(substituicoes)



In [ ]:
df_autores['aut_grau_inst'].value_counts()

In [ ]:
df_aut_grau_inst = df_autores['aut_grau_inst'].value_counts().reset_index().rename(columns={'aut_grau_inst': 'Grau de instrução', 'count': 'Total'}).sort_values(by='Grau de instrução').reset_index(drop=True)
df_aut_grau_inst['Porcentagem'] = df_aut_grau_inst.Total / df_aut_grau_inst.Total.sum()
df_aut_grau_inst.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

#### Profissão

In [ ]:
df_autores['aut_profissao'].isnull().sum()

In [ ]:
df_autores['aut_profissao'].notnull().sum()

In [ ]:
df_autores['aut_profissao'].value_counts()

In [ ]:
# Tornando como padrão os termos com mais registros
substituicoes = {
    'ADMINISTRADOR': 'ADMINISTRADOR DE EMPRESA',
    'ADVOGADO(A)': 'ADVOGADO',
    'AGRICULTOR(A)': 'AGRICULTOR',
    'APOSENTADO(A)': 'APOSENTADO',
    'CABELEIREIRO': 'CABELEIREIRA',
    'CAMAREIRO': 'CAMAREIRA',
    'COSTUREIRO(A),': 'COSTUREIRO',
    'COZINHEIRO(A),': 'COZINHEIRO',
    'ESTUDAMTE,': 'ESTUDANTE',
    'LAVRADOR(A)': 'LAVRADOR',
    'PEDAGOGO(A)': 'PEDAGOGA',
    'PEDAGOGO': 'PEDAGOGA',
    'PROMOTOR DE VENDAS': 'PROMOTORA DE VENDAS',
    'SECRETARIO(A)': 'SECRETARIA',
    'VENDEDOR(A)': 'VENDEDOR'
}

df_vitimas['vit_profissao'] = df_vitimas['vit_profissao'].replace(substituicoes)

In [ ]:
df_aut_profissao = df_vitimas['aut_profissao'].value_counts().reset_index().rename(columns={'aut_profissao': 'Profissão', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
df_aut_profissao['Porcentagem'] = df_aut_profissao.Total / df_aut_profissao.Total.sum()
df_aut_profissao.head(30).style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Grau de relacionamento

In [ ]:
df_vitimas['grau_de_relacionamento'].value_counts()

In [ ]:
# Tornando como padrão os termos com mais registros
substituicoes = {
    'nao informado': 'NAO INFORMADO',
    'Grau parentesco com o envolvido n.1-FILHO(A)': 'FILHO(A)',
    'Grau parentesco com o envolvido n.1-FILHA': 'FILHO(A)',
    'Grau parentesco com o envolvido n.1-EX_CONVIVENTE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-EX_CONJUGE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-IRMA': 'IRMAO(A)',
    'Grau parentesco com o envolvido n.1-NETO(A)': 'NETO(A)',
    'Grau parentesco com o envolvido n.1-EX CONVIVENTE': 'EX-COMPANHEIRO(A)',
    'Grau parentesco com o envolvido n.1-CASADO': 'COMPANHEIRO(A)',
	'Grau parentesco com o envolvido n.3-EX_CONVIVENTE': 'EX-COMPANHEIRO(A)',
	'Grau parentesco com o envolvido n.1-MAE': 'MAE',
	'Grau parentesco com o envolvido n.2-MAE': 'MAE',
	'Grau parentesco com o envolvido n.1-TIA': 'TIO(A)',	
	'Grau parentesco com o envolvido n.1-CONCUNHADO(A)': 'CONCUNHADO(A)',
	'Grau parentesco com o envolvido n.1-NETA': 'NETO(A)',
	'Grau parentesco com o envolvido n.1-EX SOGRA': 'SOGRO(A)',
	'Grau parentesco com o envolvido n.1-EX ESPOSA': 'EX-COMPANHEIRO(A)',
}

df_vitimas['grau_de_relacionamento'] = df_vitimas['grau_de_relacionamento'].replace(substituicoes)



In [ ]:
df_grau_relacionamento = df_vitimas['grau_de_relacionamento'].value_counts().reset_index().rename(columns={'grau_de_relacionamento': 'Grau de relacionamento', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
df_grau_relacionamento['Porcentagem'] = df_grau_relacionamento.Total / df_grau_relacionamento.Total.sum()
df_grau_relacionamento.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

### Registros

In [ ]:
df['registros'].value_counts()

In [ ]:
df_registros = df['registros'].value_counts().reset_index().rename(columns={'registros': 'Tipo', 'count': 'Total'}).sort_values(by='Total', ascending=False).reset_index(drop=True)
df_registros['Porcentagem'] = df_registros.Total / df_registros.Total.sum()
df_registros.style.format(formatter={'Porcentagem': lambda x: locale.format_string("%.2f", x*100, True) + '%' }).hide(axis="index")

In [ ]:
if 'registros_2' in df_ocorrencias.columns:
    df_ocorrencias.drop(columns=['registros_2'], inplace=True)

In [ ]:
df_ocorrencias['registros_2'] = df_ocorrencias['registros'].str.lower()

In [ ]:
df_ocorrencias['registros_2'].value_counts().head(10)

In [ ]:
import re

# Dicionário de categorias para 'registros'
categorias_registros = {
    'descumprir decisao judicial que defere medidas protetivas de urgencia':['descumprir decisao judicial que defere medidas protetivas de urgencia', 'descumprimento de medida protetiva'],
    'violencia domestica e familiar contra a mulher':['violencia domestica e familiar contra a mulher'],
    'lesao corporal':['lesao corporal'],
    'ameaca':['ameaca'],
    'estupro':['estupro'],
    'violencia sexual':['importunacao sexual', 'assedio sexual', 'intimidade sexual', 'assedio sexual', 'violacao sexual', 'abuso sexual', 'exploracao sexual', 'liberdade sexual', 'ato obsceno'],
    'tentativa de homicidio':['tentativa de homicidio'],
    'feminicidio':['feminicidio', 'femicidio', 'homicidio'],
}

# Função para substituir os valores na coluna 'registros'
def substituir_registro(valor):
    for key, values in categorias_registros.items():
        for value in values:
            if re.search(r'\b' + re.escape(value) + r'\b', valor.lower()):
                return key
    return 'outros'

In [ ]:
df_ocorrencias['registros_2'] = df_ocorrencias['registros_2'].apply(substituir_registro)

In [ ]:
df_ocorrencias['registros_2'].value_counts()

In [ ]:
df_ocorrencias[df_ocorrencias['registros_2'] == 'outros']['registros'].value_counts()